# Action potential: putting Na and K channels together

Now that we saw how to model the $Na^{+}$ and $K^{+}$ channels, we can put them together and run current clamp experiments to actually generate action potentials.

In [ ]:
# Usual imports
import sys
sys.path.append('/Users/subhasisray/src/moose-core/python')
import numpy as np
import matplotlib.pyplot as plt

import moose


# Create containers
data = moose.Neutral('/data')
model = moose.Neutral('/model')

# Create a compartment representing the squid giant axon
axon = moose.Compartment('/model/axon')
axon.Em = 0    # Hodgkin and Huxley used resting voltage as 0
axon.initVm = 0
axon.Cm = 1
axon.Rm = 1 / 0.3   # G_leak is 0.3 mS/cm^2

In [ ]:
# Create K channel and connect it to the compartment
kchan = moose.HHChannel(f'{axon.path}/K')
moose.connect(kchan, 'channel', axon, 'channel')


kchan.Ek = -12.0   # mV with respect to resting Vm
# Gbar is maximum conductance. 
# The voltage dependent conductance value is computed and 
# stored in the field Gk, which we record in a table below
kchan.Gbar = 36.0  # mS/cm^2

kchan.Xpower = 4

n_gate = moose.HHGate(f'{kchan.path}/gateX')
# gate.useInterpolation = True   # use a lookup table for alpha and beta


vdivs = 150
vmin = -30.0
vmax = 120.0

n_alpha_params = [0.1, -0.01, -1.0, -10.0, -10.0]
n_beta_params = [0.125, 0, 0, 0, 80.0]
# Note that `+` operator with lists as operands concatenates them
n_params = n_alpha_params + n_beta_params + [vdivs, vmin, vmax]
n_gate.setupAlpha(n_params)

In [ ]:
# Create Na channel and connect it to the compartment
nachan = moose.HHChannel(f'{axon.path}/Na')
moose.connect(nachan, 'channel', axon, 'channel')
nachan.Ek = 115.0   # mV with respect to resting Vm
# Gbar is maximum conductance. 
# The voltage dependent conductance value is computed and 
# stored in the field Gk, which we record in a table below
nachan.Gbar = 120.0  # mS/cm^2

nachan.Xpower = 3
nachan.Ypower = 1


m_gate = moose.HHGate(f'{nachan.path}/gateX')
m_alpha_params = [2.5, -0.1, -1.0, -25.0, -10.0]
m_beta_params = [4, 0, 0, 0, 18.0]
# Note that `+` operator with lists as operands concatenates them
m_params = m_alpha_params + m_beta_params + [vdivs, vmin, vmax]
m_gate.setupAlpha(m_params)

h_gate = moose.HHGate(f'{nachan.path}/gateY')
h_alpha_params = [0.07, 0, 0, 0, 20.0]
h_beta_params = [1, 0, 1, -30, -10.0]
# Note that `+` operator with lists as operands concatenates them
h_params = h_alpha_params + h_beta_params + [vdivs, vmin, vmax]
h_gate.setupAlpha(h_params)

In [ ]:
gNa_tab = moose.Table(f'{data.path}/gNa')
moose.connect(gNa_tab, 'requestOut', nachan, 'getGk')
gK_tab = moose.Table(f'{data.path}/gK')
moose.connect(gK_tab, 'requestOut', kchan, 'getGk')
vm_tab = moose.Table(f'{data.path}/Vm')
moose.connect(vm_tab, 'requestOut', axon, 'getVm')

In [ ]:
pulse = moose.PulseGen('/model/pulse')
moose.connect(pulse, 'output', axon, 'injectMsg')

In [ ]:
simdt = 1e-5   # simulation timestep
plotdt = 1e-3  # data collection timestep

moose.setClock(0, simdt)
moose.setClock(1, simdt)
moose.setClock(2, simdt)
moose.setClock(3, plotdt)

# moose uses `#` character as a wildcard. 
# "/model/#" gets all objects that are children of `/model`.
# "/model/#[TYPE=Compartment]" gets all `Compartment` instances that are children of `/model`.
# "/model/##" gets all objects that are descendants (children, grand children, ...) of `/model`.

moose.useClock(0, f'{model.path}/#[TYPE=Compartment]', 'init')
moose.useClock(1, f'{pulse.path}', 'process')
moose.useClock(1, f'{model.path}/#[TYPE=Compartment]', 'process')
moose.useClock(2, f'{axon.path}/#[TYPE=HHChannel]', 'process')
moose.useClock(3, f'{data.path}/#[TYPE=Table]', 'process')


In [ ]:
# Modify this to see effect of different current amplitudes
pulse.delay[0] = 10.0
pulse.level[0] = 100
pulse.width[0] = 90
pulse.baseLevel = 0

In [ ]:
runtime = 100
# `reinit` initialize the simulation 
moose.reinit()
# run the simulation for `runtime` duration
moose.start(runtime)

# Plot the voltage
plt.plot(np.arange(len(vm_tab.vector)) * vm_tab.dt, vm_tab.vector)